In [1]:
import numpy as np
import random
import os
import pandas as pd
import time
from torch.autograd import variable
import cv2
import scipy.io
import matplotlib.pyplot as plt


mat = scipy.io.loadmat('ImageData.mat')
faces = mat['SubDir_Data']
faces = np.transpose(faces)
#faces = list of ground truth 
#validation
'''
print(faces[236])
c1 = int(faces[235][2]) #xcor
c2 = int(faces[235][3]) #ycor
c3 = int(faces[235][6]) #xcor
c4 = int(faces[235][7]) #ycor
print(c1, c2, c3, c4)
'''
current = cv2.cvtColor(cv2.imread('data/images/image_0236.jpg'), cv2.COLOR_BGR2RGB)
print(current.shape)
#cv2.rectangle(current, (c1, c2), (c3, c4), (0, 255, 0), 5)
plt.imshow(current)

def last_4chars(x):
    return(x[6:-4])

facespic = os.listdir('data/images')
facespic = sorted(facespic, key = last_4chars)
#print(facespic)




/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'ImageData.mat'

In [ ]:
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

w= int(current.shape[1])
h= int(current.shape[0])
print(w,h)

b = (faces[0][2], faces[0][6], faces[0][3], faces[0][7])
bb = convert((w,h), b)
print(bb)

896 592
(0.6804391902134368, 0.48513771623915936, 0.3897425014148273, 0.8307193441319076)


In [ ]:
#to darknet format
def create_dataset(matfile, facefile):
    for i, data in enumerate(matfile):
        currentface = cv2.cvtColor(cv2.imread(f'data/images/{facefile[i]}'), cv2.COLOR_BGR2RGB)
        #since there is only faces, class will always be 1
        y, x, z = currentface.shape
        #print(x, y, z)
        class_index = 0
        #bbox_x_center = ((int(data[6]) - int(data[2]))/2)/x
        #bbox_y_center = ((int(data[7]) - int(data[3]))/2)/y
        cv2.rectangle(currentface, (int(data[2]), int(data[3])), (int(data[6]), int(data[7])), (0, 255, 0), 5)
        plt.imshow(currentface)
        #bbox_width = (int(data[6]) - int(data[2]))/x
        #bbox_height = (int(data[7]) - int(data[3]))/y
        bbox_x_center, bbox_y_center, bbox_width, bbox_height = convert((x,y), (data[2], data[6], data[3], data[7]))
        output_text = f'{class_index} {bbox_x_center} {bbox_y_center} {bbox_width} {bbox_height}'
        f = open(f'data/labels/image_{i + 1:04n}.txt', 'w')
        f.write(output_text)
        f.close
        
        

In [ ]:
#splitting data, scramble and 10-90 split
def validation(facespic):
    split = int(len(facespic) * 0.1)
    file_train = open('data/train.txt', 'w')
    file_val = open('data/val.txt', 'w')
    random.shuffle(facespic)
    for i, faces in enumerate(facespic):
        if i <= split:
            file_val.write('data/images/' + faces + '\n')
        else:
            file_train.write('data/images/' + faces + '\n')
    file_train.close()
    file_val.close()

In [ ]:
#create_dataset(faces, facespic)
#validation(facespic)
